In [ ]:
import numpy as np  
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import xgboost as xgb
import os
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [4]:
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global mode
        mode = change['new']
        print(f"Mode set to: {mode}")
mode_dropdown = widgets.Dropdown(
    options=['uncorrected', 'combat_corrected', 'scanorama_corrected'],
    value='uncorrected',
    description='Mode:',
)

display(mode_dropdown)

mode = mode_dropdown.value
print("Selected mode:", mode)
mode_dropdown.observe(on_change)

Dropdown(description='Mode:', options=('uncorrected', 'combat_corrected', 'scanorama_corrected'), value='uncor…

Selected mode: uncorrected


Mode set to: scanorama_corrected


In [ ]:
loaded_model_lineage = xgb.XGBClassifier()
loaded_model_subtyping = xgb.XGBClassifier()
loaded_model_complete = xgb.XGBClassifier()
csv_dir = '/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/QC/standard/batch_corrected_scanorama'
save_dir = f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/QC/standard/phenotyping/phenotypes_{mode}_complete'
loaded_model_complete.load_model(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/github/myeloma_standal/phenotyping/{mode}/xgboost/standard/complete/best_xgb_model_complete.json')
label_mapping_complete = pd.read_csv(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/github/myeloma_standal/phenotyping/{mode}/xgboost/standard/complete/phenotype_label_mapping.csv')

In [ ]:
print(f"Running in {mode} mode")
n = 0
for csv in os.listdir(csv_dir):
    if not csv.endswith('.csv'):
        print(f"Skipping {csv}")
        continue
    print(f"Processing {csv}")
    n += 1
    csv_path = os.path.join(csv_dir, csv)
    table = pd.read_csv(os.path.join(csv_path))
    table.rename(columns={'centroid-0': 'Y_centroid', 'centroid-1': 'X_centroid', 'Object': 'CellID'}, inplace=True)
    image_ID = csv_path.split('/')[-1].split('.')[0]
    table['image_ID'] = image_ID
    table['index'] = table['image_ID'] + '_' + table['CellID'].astype(str)
    table.set_index('index', inplace=True)
    table = table[table['area'] >= 4]

    original_columns = table.columns
    cols_to_add = table[['HistoneH3', 'CD98','X_centroid', 'Y_centroid', 'CellID', 'image_ID', " 1", " 2", " 3", " 4", " 5", " 6", "191Ir", "193Ir"]]
    table.drop(columns=[" 1", " 2", " 3", " 4", " 5", " 6", "191Ir", "193Ir", 'HistoneH3', 'CD98','X_centroid', 'Y_centroid', 'CellID', 'image_ID'], inplace=True)

    if mode == 'uncorrected':
        transformed = np.arcsinh(table.iloc[:, 0:32])
        table.drop(columns=table.columns[0:32], inplace=True)
        phenotypes = pd.concat([transformed, table], axis=1)

        transformed = np.arcsinh(cols_to_add.iloc[:, 0:2])
        cols_to_add.drop(columns=cols_to_add.columns[0:2], inplace=True)
        cols_to_add = pd.concat([transformed, cols_to_add], axis=1)
    else:
        phenotypes = table
    

    complete_predictions = loaded_model_complete.predict(phenotypes)
    phenotypes['predicted_label'] = complete_predictions

    phenotypes.reset_index('index', inplace=True)
    phenotypes = pd.merge(phenotypes, label_mapping_complete, left_on='predicted_label', right_on='EncodedLabel', how='left')
    phenotypes.set_index('index', inplace=True)
    result = pd.concat([phenotypes, cols_to_add], axis=1)
    result.drop(columns=['EncodedLabel', 'predicted_label'], inplace=True)

    new_columns = [col for col in result.columns if col not in original_columns]
    original_columns = list(original_columns)
    original_columns.extend(new_columns)
    result = result[original_columns]
    result.to_csv(os.path.join(save_dir, csv), index=False)
print(f"Processed and saved {n} files")

Running in scanorama_corrected mode
Processing TS-373_IMC77_B_001.csv
Processing TS-373_IMC03_MGUS_002.csv
Processing TS-373_IMC14_B_002.csv
Processing TS-373_IMC04_MGUS_002.csv
Processing TS-373_IMC72_MGUS_001.csv
Processing TS-373_IMC12_UB_001.csv
Processing TS-373_IMC86_B_002.csv
Processing TS-373_IMC42_B_001.csv
Processing TS-373_IMC35_B_002.csv
Processing TS-373_IMC38_B_001.csv
Processing TS-373_IMC90.1_B_001.csv
Processing TS-373_IMC56_B_001.csv
Processing TS-373_IMC92_B_002.csv
Processing TS-373_IMC37-1_MGUS_001.csv
Processing TS-373_IMC77_B_002.csv
Processing TS-373_IMC03_MGUS_001.csv
Processing TS-373_IMC14_B_001.csv
Processing TS-373_IMC04_MGUS_001.csv
Processing TS-373_IMC12_UB_002.csv
Processing TS-373_IMC72_MGUS_002.csv
Processing TS-373_IMC86_B_001.csv
Processing TS-373_IMC35_B_001.csv
Processing TS-373_IMC38_B_002.csv
Processing TS-373_IMC90.1_B_002.csv
Processing TS-373_IMC56_B_002.csv
Processing TS-373_IMC92_B_001.csv
Processing TS-373_IMC33_B_002.csv
Processing TS-373